**  LOGISTIC REGRESSION TO  PREDICT HEART DISEASE.**

 **Introduction**
 
World Health Organization has estimated 12 million deaths occur worldwide, every year due to Heart diseases. Half the deaths in the United States and other developed countries are due to cardio vascular diseases. The early prognosis of cardiovascular diseases can aid in making decisions on lifestyle changes in high risk patients and in turn reduce the complications. This research intends to pinpoint the most relevant/risk factors of heart disease as well as predict the overall risk using logistic regression.

** # Supress Warnings**

In [ ]:


import warnings
warnings.filterwarnings('ignore')

**import libraries**

In [ ]:

import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import matplotlib.mlab as mlab

#Importing the dataset and dropping the unwanted columns 
heart_df=pd.read_csv("../input/framingham.csv")
heart_df.drop(['education'],axis=1,inplace=True)
heart_df.head()


Data Preparation
Source:
The dataset is publically available on the Kaggle website, and it is from an ongoing ongoing cardiovascular study on residents of the town of Framingham, Massachusetts. The classification goal is to predict whether the patient has 10-year risk of future coronary heart disease (CHD).The dataset provides the patients’ information. It includes over 4,000 records and 15 attributes

In [ ]:
#Renaming the column name
heart_df.rename(columns={'male':'Sex_male'},inplace=True)

heart_df.head()

Missing values

In [ ]:
# Checking the missing values 
heart_df.isnull().sum()

Exploratory Analysis

In [ ]:
#Counting the missing values and dropping them
count=0
for i in heart_df.isnull().sum(axis=1):
    if i>0:
        count=count+1
print('Total number of rows with missing values is ', count)
print('since it is only',round((count/len(heart_df.index))*100), 'percent of the entire dataset the rows with missing values are excluded.')

heart_df.dropna(axis=0,inplace=True)

heart_df.describe()

**again checking null values**

In [ ]:
heart_df.isnull().sum()

In [ ]:
#Adding a constant
from statsmodels.tools import add_constant as add_constant
heart_df_constant = add_constant(heart_df)
heart_df_constant.head()


**Logistic Regression**

Logistic regression is a type of regression analysis in statistics used for prediction of outcome of a categorical dependent variable from a set of predictor or independent variables. In logistic regression the dependent variable is always binary. Logistic regression is mainly used to for prediction and also calculating the probability of success.

**Chi-square Test**

● A Chi-square Test (also written 𝜒2) is used to determine the probability of an observed frequency of events given an expected frequency

**Chi-square Test**

● The chi-square formula considers the sum of square distances between observed values O and expected values E, divided by each expected value:
𝜒2 = ((𝑂 − 𝐸)e2)/E

In [ ]:


st.chisqprob = lambda chisq, df: st.chi2.sf(chisq, df)
cols=heart_df_constant.columns[:-1]
model=sm.Logit(heart_df.TenYearCHD,heart_df_constant[cols])
result=model.fit()
result.summary()


**#Feature Selection: Backward elemination (P-value approach)
#Takes in the dataframe, the dependent variable and a list of column names, 
#runs the regression repeatedly eleminating feature with the highest
#P-value above alpha one at a time and returns the regression summary 
#with all p-values below alpha
**

In [ ]:

def back_feature_elem (data_frame,dep_var,col_list):
    while len(col_list)>0 :
        model=sm.Logit(dep_var,data_frame[col_list])
        result=model.fit(disp=0)
        largest_pvalue=round(result.pvalues,3).nlargest(1)
        if largest_pvalue[0]<(0.05):
            return result
            break
        else:
            col_list=col_list.drop(largest_pvalue.index)

result=back_feature_elem(heart_df_constant,heart_df.TenYearCHD,cols)


In [ ]:
#Interpreting the results: Odds Ratio, Confidence Intervals and Pvalues

params = np.exp(result.params)
conf = np.exp(result.conf_int())
conf['OR'] = params
pvalue=round(result.pvalues,3)
conf['pvalue']=pvalue
conf.columns = ['CI 95%(2.5%)', 'CI 95%(97.5%)', 'Odds Ratio','pvalue']
print ((conf))


**#Splitting data to train and test split**

In [ ]:

import sklearn
new_features=heart_df[['age','Sex_male','cigsPerDay','totChol','sysBP','glucose','TenYearCHD']]
x=new_features.iloc[:,:-1]
y=new_features.iloc[:,-1]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=5)

from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()
logreg.fit(x_train,y_train)
y_pred=logreg.predict(x_test)


**#Confusion matrix**

In [ ]:

from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(y_test,y_pred)
print(confusion_matrix)

**#Model accuracy**

In [ ]:

sklearn.metrics.accuracy_score(y_test,y_pred)
